# Data exploration

In [10]:
import pandas as pd 
import sys 
import os 
import json 
from pathlib import Path 
import ujson

In [5]:
fpath =  "/home/INM705/INM705_CW_Collins_Velagala/Datasets/coco/annotations/"
#lvis_v1_val.json" #"/home/Datasets/coco/annotations/"

## Validation set 

In [39]:
stage = "train"
fname = "lvis_v1_{}.json".format(stage)
#p = Path('/home/Datasets/coco/annotations')
#list(p.iterdir())

In [40]:
file = f = open (fpath + fname, "r")
data = ujson.loads(f.read())

In [41]:
len(data['images'])

100170

In [15]:
type(data['images'][0])#['height']['width']

dict

In [16]:
data['images'][0]

{'date_captured': '2013-11-14 17:02:52',
 'neg_category_ids': [279, 899, 127, 180, 1136, 725, 663],
 'id': 397133,
 'license': 4,
 'height': 427,
 'width': 640,
 'flickr_url': 'http://farm7.staticflickr.com/6116/6255196340_da26cf2c9e_z.jpg',
 'coco_url': 'http://images.cocodataset.org/val2017/000000397133.jpg',
 'not_exhaustive_category_ids': [914, 801, 566, 139, 1021]}

In [18]:
data['images'][0]['id', 'height', 'width']

KeyError: ('id', 'height', 'width')

In [42]:
img_size_df = pd.DataFrame(columns = ["id" , "height", "width"] )
keys = ['id', 'height', 'width']

for i in range(len(data['images'])):
    img_data = data['images'][i] 
    values = list(map(img_data.get, keys))
    temp = pd.DataFrame({keys[0] : [values[0]],
                         keys[1] : [values[1]],
                         keys[2] : [values[2]]
                        })
    img_size_df = pd.concat([img_size_df, temp], axis = 0) 
    #if i ==2: break                    
    #img, h, w = data
    
temp
img_size_df

,id,height,width
0,391895,360,640
0,522418,480,640
0,184613,336,500
0,318219,640,556
0,554625,640,426
...,...,...,...
0,444010,480,640
0,565004,427,640
0,516168,480,640
0,547503,375,500


In [43]:
img_size_df['dim'] = img_size_df['height'].astype(str) + 'x' + img_size_df['width'].astype(str)
img_size_df.head()

,id,height,width,dim
0,391895,360,640,360x640
0,522418,480,640,480x640
0,184613,336,500,336x500
0,318219,640,556,640x556
0,554625,640,426,640x426


In [44]:
#train set  
img_size_df.groupby(['dim']).count().sort_values(by = ['id'], ascending = False)

,id,height,width
dim,,,
480x640,21721,21721,21721
427x640,12353,12353,12353
640x480,7112,7112,7112
426x640,4572,4572,4572
375x500,4382,4382,4382
...,...,...,...
409x286,1,1,1
408x575,1,1,1
408x544,1,1,1


In [38]:
#validation_set 
img_size_df.groupby(['dim']).count().sort_values(by = ['id'], ascending = False)

,id,height,width
dim,,,
480x640,4173,4173,4173
427x640,2352,2352,2352
640x480,1414,1414,1414
375x500,920,920,920
426x640,840,840,840
...,...,...,...
450x620,1,1,1
451x300,1,1,1
453x600,1,1,1


In [5]:
cat_val_df = pd.DataFrame(columns = ["idx", "name", "image_count", "instance_count", "frequency"])

for cat in data['categories']: 
    row = {"idx" : cat['id'],
        "name": cat['name'],
        "image_count": cat['image_count'],
        "instance_count": cat["instance_count"],
        "frequency" : cat["frequency"]}
    cat_val_df = cat_val_df.append(row, ignore_index = True)
    #break

In [6]:
cat_val_df.head()

,idx,name,image_count,instance_count,frequency
0,1,aerosol_can,8,11,c
1,2,air_conditioner,65,146,f
2,3,airplane,349,619,f
3,4,alarm_clock,25,60,f
4,5,alcohol,14,149,c


In [7]:
cat_val_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1203 entries, 0 to 1202
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   idx             1203 non-null   object
 1   name            1203 non-null   object
 2   image_count     1203 non-null   object
 3   instance_count  1203 non-null   object
 4   frequency       1203 non-null   object
dtypes: object(5)
memory usage: 47.1+ KB


In [125]:
cat_val_df['image_count'] = pd.to_numeric(cat_val_df['image_count'] )
cat_val_df['instance_count'] = pd.to_numeric(cat_val_df['instance_count'] )
cat_val_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1203 entries, 0 to 1202
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   idx             1203 non-null   object
 1   name            1203 non-null   object
 2   image_count     1203 non-null   int64 
 3   instance_count  1203 non-null   int64 
 4   frequency       1203 non-null   object
dtypes: int64(2), object(3)
memory usage: 47.1+ KB


In [126]:
cat_val_df.groupby('frequency').count()

,idx,name,image_count,instance_count
frequency,,,,
c,461,461,461,461
f,405,405,405,405
r,337,337,337,337


In [127]:
cat_val_df.groupby('frequency').describe()

image_count                                                          \
                count        mean         std  min   25%    50%    75%    max   
frequency                                                                       
c               461.0    7.874187    5.245433  0.0   4.0    7.0   11.0   30.0   
f               405.0  163.402469  135.300229  5.0  43.0  116.0  297.0  414.0   
r               337.0    0.982196    1.281616  0.0   0.0    1.0    2.0    7.0   

          instance_count                                                   \
                   count        mean         std  min   25%    50%    75%   
frequency                                                                   
c                  461.0   28.373102   49.910656  0.0   6.0   13.0   27.0   
f                  405.0  568.955556  804.574910  8.0  96.0  298.0  735.0   
r                  337.0    3.560831   15.067803  0.0   0.0    1.0    2.0   

                   
              max  
frequency          
c           467.0  
f          9156.0  
r           242.0

## Training set 

In [99]:
stage = "train"
fname = "lvis_v1_{}.json".format(stage)

In [100]:
f = open (fpath + fname, "r")
data = json.loads(f.read())

In [104]:
cat_df = pd.DataFrame(columns = ["idx", "name", "image_count", "instance_count", "frequency"])

for cat in data['categories']: 
    row = {"idx" : cat['id'],
        "name": cat['name'],
        "image_count": cat['image_count'],
        "instance_count": cat["instance_count"],
        "frequency" : cat["frequency"]}
    cat_df = cat_df.append(row, ignore_index = True)
    #break
cat_df['image_count'] = pd.to_numeric(cat_df['image_count'] )
cat_df['instance_count'] = pd.to_numeric(cat_df['instance_count'] )
cat_df.head()

,idx,name,image_count,instance_count,frequency
0,1,aerosol_can,64,109,c
1,2,air_conditioner,364,1081,f
2,3,airplane,1911,3720,f
3,4,alarm_clock,149,158,f
4,5,alcohol,29,207,c


In [105]:
cat_df.groupby('frequency').count()

,idx,name,image_count,instance_count
frequency,,,,
c,461,461,461,461
f,405,405,405,405
r,337,337,337,337


In [107]:
cat_df.groupby('frequency').sum()

,image_count,instance_count
frequency,,
c,17998,64626
f,341458,1200258
r,1496,5257


In [106]:
cat_df.groupby('frequency').describe()

image_count                                                       \
                count        mean         std    min    25%    50%     75%   
frequency                                                                    
c               461.0   39.041215   23.908622   11.0   18.0   32.0    54.0   
f               405.0  843.106173  691.088785  101.0  218.0  588.0  1582.0   
r               337.0    4.439169    2.914967    1.0    2.0    4.0     7.0   

                  instance_count                                          \
              max          count         mean          std    min    25%   
frequency                                                                  
c           100.0          461.0   140.186551   221.141623   11.0   38.0   
f          1977.0          405.0  2963.600000  4193.132864  114.0  574.0   
r            10.0          337.0    15.599407    32.542414    1.0    3.0   

                                    
              50%     75%      max  
frequency                           
c            71.0   140.0   1869.0  
f          1595.0  3960.0  50552.0  
r             7.0    14.0    333.0

In [109]:
cat_df[cat_df.frequency == "c"]

,idx,name,image_count,instance_count,frequency
0,1,aerosol_can,64,109,c
4,5,alcohol,29,207,c
5,6,alligator,26,39,c
6,7,almond,59,1700,c
7,8,ambulance,22,25,c
...,...,...,...,...,...
1195,1196,wrench,32,80,c
1198,1199,yacht,12,50,c
1199,1200,yogurt,52,116,c
1200,1201,yoke_(animal_equipment),11,20,c


In [110]:
cat_df[cat_df.frequency == "f"]

,idx,name,image_count,instance_count,frequency
1,2,air_conditioner,364,1081,f
2,3,airplane,1911,3720,f
3,4,alarm_clock,149,158,f
10,11,antenna,505,1018,f
11,12,apple,1207,17451,f
...,...,...,...,...,...
1189,1190,wineglass,941,4259,f
1190,1191,blinder_(for_horses),113,271,f
1196,1197,wristband,128,268,f
1197,1198,wristlet,623,1330,f


In [117]:
cat_df[cat_df.frequency == "r"].sort_values(by = ["instance_count"], ascending = False)

,idx,name,image_count,instance_count,frequency
637,638,legume,10,333,r
786,787,pennant,8,235,r
754,755,papaya,10,206,r
808,809,pinwheel,3,172,r
434,435,fig_(fruit),4,147,r
...,...,...,...,...,...
971,972,sling_(bandage),1,1,r
209,210,car_battery,1,1,r
1004,1005,stagecoach,1,1,r
1009,1010,steak_knife,1,1,r


In [130]:
cat_val_df[cat_val_df.name.isin(["zucchini", "papaya", "apple"])]

,idx,name,image_count,instance_count,frequency
11,12,apple,210,3116,f
754,755,papaya,7,96,r
1202,1203,zucchini,17,178,c
